In [1]:
from qutip import *
# import for interactive plot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# import for plot   
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})

width= 5
aspect_ratio= 1.13
fig_size = (width, width / aspect_ratio)
plt.figure(figsize=fig_size, constrained_layout=True)

<Figure size 360x318.584 with 0 Axes>

<Figure size 360x318.584 with 0 Axes>

In [2]:

eV = 1.0 / 27.2114
cmn1 = 1.0/219474.63

def Vnew(R, wb= 0.02, Eb = 0.4):
    
    wb = wb * cmn1
    Eb = Eb * cmn1
    a1 = wb*wb/2.0
    a2 = a1*a1/(4*Eb)
    V  = -a1*R**2 + a2*R**4

    V0 = np.min(V)
    return - V0 + V


# Kinetic energy for electron 
def Te(re):
 N = float(len(re))
 mass = 1.0
 Tij = np.zeros((int(N),int(N)))
 Rmin = float(re[0])
 Rmax = float(re[-1])
 step = float((Rmax-Rmin)/N)
 K = np.pi/step

 for ri in range(int(N)):
  for rj in range(int(N)):
    if ri == rj:  
     Tij[ri,ri] = (0.5/mass)*K**2.0/3.0*(1+(2.0/N**2)) 
    else:    
     Tij[ri,rj] = (0.5/mass)*(2*K**2.0/(N**2.0))*((-1)**(rj-ri)/(np.sin(np.pi*(rj-ri)/N)**2)) 
 return Tij

def HO(R, R0,w):
    return 0.5 * (R-R0)**2 * w**2



wb  = 1000 
Eb  = 2250
nR  = 1024
Rmax = 100
R = np.linspace(-Rmax,Rmax, nR)
V = Vnew(R,wb,Eb) 

Hij = Te(R) + np.diag(V)
Ei, Um   = np.linalg.eigh(Hij)

Ψ1 = Um[:,0]
Ψ2 = Um[:,1]

Φ1 = (Ψ1 + Ψ2)/np.sqrt(2)
Φ2 = (Ψ1 - Ψ2)/np.sqrt(2)



# Environment
### $\gamma$ determines how strongly the environment is coupled to the system 


In [21]:

# t is a playable slider
# a lindblad code using qutip
@interact(Δ = (0, 10, 0.1), γ = (0, 1, 0.1), t = (0, 10, 0.01) )
def TLS(Δ = 1, γ = 0.0, t = 0):
    # Δ = 1
    ΔE = 0.0
    # γ = 0.1
    β  = 0.0
    H = sigmax() * Δ + sigmaz() * ΔE / 2 
    
    # set figure size
    width= 8
    aspect_ratio= 2.6
    fig_size = (width, width / aspect_ratio)
    plt.figure(figsize=fig_size)
    
    γ1 = γ
    γ2 = γ1 * np.exp(- β * ΔE)
    c_ops = [np.sqrt(γ2) * sigmam(),np.sqrt(γ1) * sigmam().dag()]
    
    tlist = np.linspace(0, 10, 1000)
    dt = tlist[1] - tlist[0]
    psi0 = basis(2, 0)
    # define a density matrix
    rho11 = basis(2, 0) * basis(2, 0).dag() 
    rho22 = basis(2, 1) * basis(2, 1).dag()
    rho12 = basis(2, 0) * basis(2, 1).dag() 
    rho21 = basis(2, 1) * basis(2, 0).dag() 
    result = mesolve(H, psi0, tlist, c_ops, [rho11,rho22, rho12, rho21])
    
    # plot a figure on the left
    plt.subplot(121)
    plt.plot(tlist, result.expect[0], label = '$\\rho_{11}$', color = '#ef5777')
    plt.plot(tlist, result.expect[1], label = '$\\rho_{22}$', color = '#48dbfb')
    plt.scatter(t, result.expect[0][int(t/dt)], color = 'red')
    plt.scatter(t, result.expect[1][int(t/dt)], color = 'blue')
    # legend left middle
    plt.legend(loc = 'center left')
    
    plt.xlim(0, 10)
    plt.ylim(-0.05, 1.05)
    plt.xlabel("Time")
    # plot a figure on the right
    plt.subplot(122)
    
    color = ['#ef5777', '#48dbfb', '#8854d0', '#079992', '#e58e26',"#ff9ff3"]
    plt.plot(R,  V/cmn1, c =  "#000000", lw = 3)
    plt.plot(R, Φ1*0 + Ei[0]/cmn1, c =  "#000000")
    
    pij = np.zeros((2,2), dtype = np.complex128)
    pij[0,0] = result.expect[0][int(t/dt)]
    pij[1,1] = result.expect[1][int(t/dt)]
    pij[0,1] = result.expect[2][int(t/dt)]
    pij[1,0] = result.expect[3][int(t/dt)]
    
    # density along x
    Px =  pij[0,0] * Φ1**2 + pij[1,1] * Φ2**2  +   pij[0,1] * Φ1 * Φ2 + pij[1,0] * Φ2 * Φ1
    
    plt.plot(R, Px.real * 1E5 + Ei[0]/cmn1, c =  color[0], ls = '-')
    plt.fill_between(R,Ei[0]/cmn1,  Px.real * 1E5 + Ei[0]/cmn1, facecolor=color[0], alpha = 0.3)
    
    plt.title("t = %.2f" %t)
    plt.ylabel("$|\Psi(x)|^2$")
    plt.xlabel("x")
    plt.ylim(0,3000)
    # put space between two figures
    plt.subplots_adjust(wspace=0.5)
    

interactive(children=(FloatSlider(value=1.0, description='Δ', max=10.0), FloatSlider(value=0.0, description='γ…

## Play with energy gap ($\Delta E$), Temperature ($\beta$)


In [22]:

# t is a playable slider
# a lindblad code using qutip
@interact(Δ = (0, 10, 0.1), ΔE = (0,5,0.1), γ = (0, 1, 0.1), β  = (0, 5, 0.1))
def TLS(Δ = 1, ΔE = 0, γ = 0.0, β = 5):
    # Δ = 1
    # dE = 0.0
    # γ = 0.1
    H = sigmax() * Δ + sigmaz() * ΔE / 2 
    
    # set figure size
    width= 8
    aspect_ratio= 2.6
    fig_size = (width, width / aspect_ratio)
    plt.figure(figsize=fig_size)
    
    γ1 = γ
    γ2 = γ1 * np.exp(- β * ΔE)
    c_ops = [np.sqrt(γ2) * sigmam(),np.sqrt(γ1) * sigmam().dag()]
    
    tlist = np.linspace(0, 10, 1000)
    dt = tlist[1] - tlist[0]
    psi0 = basis(2, 0)
    # define a density matrix
    rho11 = basis(2, 0) * basis(2, 0).dag() 
    rho22 = basis(2, 1) * basis(2, 1).dag()
    rho12 = basis(2, 0) * basis(2, 1).dag() 
    rho21 = basis(2, 1) * basis(2, 0).dag() 
    result = mesolve(H, psi0, tlist, c_ops, [rho11,rho22, rho12, rho21])
    
    # plot a figure on the left

    plt.plot(tlist, result.expect[0], label = '$\\rho_{11}$', color = '#ef5777')
    plt.plot(tlist, result.expect[1], label = '$\\rho_{22}$', color = '#48dbfb')
    # legend outside the plot (above)
    
    plt.xlim(0, 10)
    plt.ylim(-0.05, 1.05)
    plt.legend(loc = 'center left')
    plt.xlabel("Time")

interactive(children=(FloatSlider(value=1.0, description='Δ', max=10.0), FloatSlider(value=0.0, description='Δ…